In [1]:
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 833.4 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 42.4 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 71.5 MB/s eta 0:00:00:00:01
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.33.1
    Uninstalling huggingface-hub-0.33.1:
      Successfully uninstalled huggingface-hub-0.33.1
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.4
    Uninstalling transformers-4.52.4:
      Successfully uninstalled transformers-4.52.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavi

In [13]:
import csv
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from IPython.display import FileLink
import re

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM

In [3]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-3-mini-4k-instruct") # model for rectification
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-3-mini-4k-instruct", torch_dtype=torch.float16, device_map="auto").eval()

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [10]:
# this is an instruct model, works better with the chat functionality, heavier than commit model, but hopefully better
def build_prompt(message: str, filename: str, inference: str) -> str: # wasn't working in single shot, need to do this
    messages = [
        {
            "role": "system",
            "content": (
                "You are a senior engineer. Rewrite commit messages as a single, "
                "concise, imperative one-liner (<= 80 chars). "
                "Do NOT include explanations, prefixes, or extra lines."
            ),
        },
        {
            "role": "user",
            "content": (
                "Example:\n"
                "Message: fixed bug in login function\n"
                "Filename: auth.py\n"
                "Fix type: bugfix\n"
                "Output: fix login bug in auth.py\n\n"
                f"Now process:\n"
                f"Message: {message}\n"
                f"Filename: {filename}\n"
                f"Fix type: {inference}\n"
                "Output ONLY the final commit message."
            ),
        },
    ]
    return tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

def postprocess(text: str) -> str:
    t = text.strip()
    t = re.sub(r'(?i)^create a (new )?commit message.*?:\s*', '', t).strip() # it was echoing this sentence hence
    for line in t.splitlines(): # get only first non empty line, it was giving multiple lines, with repetition hence
        if line.strip():
            return line.strip()
    return t

@torch.inference_mode()
def rectify_commit(message: str, filename: str, inference: str) -> str:
    prompt = build_prompt(message, filename, inference)
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(model.device) # jic, because gpu

    outputs = model.generate(
        **inputs,
        max_new_tokens=64,
        do_sample=False,
        num_beams=4,
        no_repeat_ngram_size=3, # don't repeat, was doing this earlier
        repetition_penalty=1.05, # same as above
        eos_token_id=tokenizer.eos_token_id,
    )

    gen_ids = outputs[0][inputs["input_ids"].shape[-1]:]  # cut off prompt
    text = tokenizer.decode(gen_ids, skip_special_tokens=True)
    return postprocess(text)

In [14]:
input_file = "/kaggle/input/repository-commit-data/input_for_rectification.csv"
output_file = "rectified_commits.csv"

count = 0  # total rows 513

with open(input_file, 'r', encoding="utf-8") as f_in, \
     open(output_file, 'w', newline='', encoding="utf-8") as f_out:
    
    reader = csv.DictReader(f_in)
    writer = csv.writer(f_out)
    
    writer.writerow(reader.fieldnames + ['Rectified Message']) # add col rectified msg
    
    for row in reader:
        rect_msg = rectify_commit(message=row['Message'], filename=row['Filename'], inference=row['LLM Inference (fix type)'])
        
        writer.writerow([row[col] for col in reader.fieldnames] + [rect_msg])

        count += 1
        if count % 10 == 0:
            print(f"Row {count}:")
            print(f"Message: {row['Message']}\nFilename: {row['Filename']}\nLLM Inference: {row['LLM Inference (fix type)']}")
            print(f"Rectified message: {rect_msg}\n")

        torch.cuda.empty_cache()

Row 10:
Message: signal_stoploss_u_b_1: add Elder Ray Index check
Filename: NostalgiaForInfinityNext.py
LLM Inference: add nostalgia for infinity next
Rectified message: add elder ray index check in nostalgic infinity next

Row 20:
Message: Also try `resolve()`'ed path. Log that `hold-trades.json` was not found
Filename: NostalgiaForInfinityNext.py
LLM Inference: fix nostalgiaforinfinitynext
Rectified message: fix resolve path, log hold trades not found

Row 30:
Message: The exchange data now comes from https://github.com/iterativv/NostalgiaForInfinityData
Filename: user_data\backtesting-kucoin.env
LLM Inference: update exchange_data.rb
Rectified message: update exchange data in backtesting.env

Row 40:
Message: Add Emulated Backtest AgeFilter logic. Update exchange downtime protection (data validity).
Filename: NostalgiaForInfinityNext.py
LLM Inference: update nostalgiaforinfinitynext.py
Rectified message: add emulated backtest age filter and update exchange data validity

Row 50:
Mes

In [16]:
display(FileLink("rectified_commits.csv"))

/kaggle/working/rectified_commits.csv

In [15]:
# This didn't work, single shot makes it echo the input, repeat itself and not understand the task
# input_file = "/kaggle/input/repository-commit-data/input_for_rectification.csv"
# output_file = "rectified_commits.csv"

# count = 0 # total 513

# with open(input_file, 'r', encoding="utf-8") as f_in, \
#      open(output_file, 'w', newline='', encoding="utf-8") as f_out:
    
#     reader = csv.DictReader(f_in)
#     writer = csv.writer(f_out)
    
#     # Write headers
#     writer.writerow(reader.fieldnames + ['Rectified Message'])
    
#     for row in reader:
#         prompt = f"Create a new commit message, this is the information known:\nMessage1: {row['Message']}\nMessage 2: {row['LLM Inference (fix type)']}Filename: {row['Filename']}"
        
#         input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)
#         output_ids = model.generate(input_ids, max_length=150, num_beams=5)
#         rectified_message = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        
#         writer.writerow([row[col] for col in reader.fieldnames] + [rectified_message])

#         count+=1
#         if count%10==0:
#             print(f"Row {count}:")
#             print(f"Message: {row['Message']}\nFilename: {row['Filename']}\nLLM Inference: {row['LLM Inference (fix type)']}")
#             print(f"Rectified message: {rectified_message}\n")
        
#         del input_ids, output_ids # clearing gpu memory jic
#         torch.cuda.empty_cache()
# display(FileName("rectified_commits.csv"))